In [1]:
import numpy as np
import optuna
import lightgbm as lgb
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split

In [2]:
# 二分类问题
data, target = sklearn.datasets.load_breast_cancer(return_X_y=True)
print(data.shape)

(569, 30)


In [3]:
# 第一个参数必须为trial
def objective(trial, data, target):
    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25)
    dtrain = lgb.Dataset(train_x, label=train_y)
    dvalid = lgb.Dataset(valid_x, label=valid_y)

    param = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }

    # 也可在次进行交叉验证
    es_func = lgb.early_stopping(stopping_rounds=200)
    gbm = lgb.train(param, dtrain,
                    valid_sets=[dvalid],
                    callbacks=[es_func])
    preds = gbm.predict(valid_x)
    pred_labels = np.around(preds)
    accuracy = sklearn.metrics.accuracy_score(valid_y, pred_labels)
    return accuracy

In [4]:
study = optuna.create_study(direction="maximize")
func = lambda trial: objective(trial, data, target)
study.optimize(func, n_trials=100)

[I 2022-06-16 18:56:34,385] A new study created in memory with name: no-name-f614eaae-3751-4111-9341-22b5fd16ab2c
[I 2022-06-16 18:56:34,456] Trial 0 finished with value: 0.965034965034965 and parameters: {'lambda_l1': 2.1148929489558815e-08, 'lambda_l2': 1.0278205177422657e-07, 'num_leaves': 204, 'feature_fraction': 0.7631282496382137, 'bagging_fraction': 0.857726565842192, 'bagging_freq': 6, 'min_child_samples': 57}. Best is trial 0 with value: 0.965034965034965.
[I 2022-06-16 18:56:34,512] Trial 1 finished with value: 0.958041958041958 and parameters: {'lambda_l1': 0.4000052768495882, 'lambda_l2': 4.644368885510678e-05, 'num_leaves': 108, 'feature_fraction': 0.6049632071698764, 'bagging_fraction': 0.6878962217203524, 'bagging_freq': 1, 'min_child_samples': 59}. Best is trial 0 with value: 0.965034965034965.


Training until validation scores don't improve for 200 rounds
Did not meet early stopping. Best iteration is:
[54]	valid_0's binary_logloss: 0.117112
Training until validation scores don't improve for 200 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's binary_logloss: 0.099495
Training until validation scores don't improve for 200 rounds
Did not meet early stopping. Best iteration is:
[80]	valid_0's binary_logloss: 0.0875114

[I 2022-06-16 18:56:34,594] Trial 2 finished with value: 0.972027972027972 and parameters: {'lambda_l1': 5.987897185359307e-05, 'lambda_l2': 1.0165528300781366e-08, 'num_leaves': 95, 'feature_fraction': 0.947350310150389, 'bagging_fraction': 0.8790491610616912, 'bagging_freq': 4, 'min_child_samples': 33}. Best is trial 2 with value: 0.972027972027972.
[I 2022-06-16 18:56:34,642] Trial 3 finished with value: 0.9790209790209791 and parameters: {'lambda_l1': 0.00031612052696870855, 'lambda_l2': 0.015511824241716148, 'num_leaves': 168, 'feature_fraction': 0.46123245125163287, 'bagging_fraction': 0.610985350942228, 'bagging_freq': 7, 'min_child_samples': 71}. Best is trial 3 with value: 0.9790209790209791.
[I 2022-06-16 18:56:34,689] Trial 4 finished with value: 0.965034965034965 and parameters: {'lambda_l1': 0.00020141206006496054, 'lambda_l2': 5.446481113041442, 'num_leaves': 21, 'feature_fraction': 0.9843428904060839, 'bagging_fraction': 0.57939311537709, 'bagging_freq': 7, 'min_child_sa


Training until validation scores don't improve for 200 rounds
Did not meet early stopping. Best iteration is:
[91]	valid_0's binary_logloss: 0.0629366
Training until validation scores don't improve for 200 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's binary_logloss: 0.107235
Training until validation scores don't improve for 200 rounds
Did not meet early stopping. Best iteration is:
[86]	valid_0's binary_logloss: 0.109807
Training until validation scores don't improve for 200 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's binary_logloss: 0.103018


[I 2022-06-16 18:56:34,797] Trial 6 finished with value: 0.965034965034965 and parameters: {'lambda_l1': 5.064156642055945, 'lambda_l2': 0.2722233935554987, 'num_leaves': 161, 'feature_fraction': 0.7015131912780671, 'bagging_fraction': 0.8454458291227431, 'bagging_freq': 7, 'min_child_samples': 85}. Best is trial 3 with value: 0.9790209790209791.
[I 2022-06-16 18:56:34,835] Trial 7 finished with value: 0.9790209790209791 and parameters: {'lambda_l1': 0.0038784705784339494, 'lambda_l2': 8.05669359917271e-05, 'num_leaves': 71, 'feature_fraction': 0.48766917240380553, 'bagging_fraction': 0.4066214832361069, 'bagging_freq': 5, 'min_child_samples': 48}. Best is trial 3 with value: 0.9790209790209791.
[I 2022-06-16 18:56:34,941] Trial 8 finished with value: 0.993006993006993 and parameters: {'lambda_l1': 1.6084068381872842e-08, 'lambda_l2': 6.62576376003583e-05, 'num_leaves': 200, 'feature_fraction': 0.9973325730249161, 'bagging_fraction': 0.8025525224526386, 'bagging_freq': 1, 'min_child_sa

...

Did not meet early stopping. Best iteration is:
[70]	valid_0's binary_logloss: 0.09038
Training until validation scores don't improve for 200 rounds
Did not meet early stopping. Best iteration is:
[76]	valid_0's binary_logloss: 0.138154
Training until validation scores don't improve for 200 rounds


[I 2022-06-16 18:56:43,317] Trial 97 finished with value: 0.993006993006993 and parameters: {'lambda_l1': 0.00023157645073283332, 'lambda_l2': 2.3339965955416078e-08, 'num_leaves': 108, 'feature_fraction': 0.4957086351839552, 'bagging_fraction': 0.8952517508681361, 'bagging_freq': 1, 'min_child_samples': 8}. Best is trial 39 with value: 1.0.
[I 2022-06-16 18:56:43,470] Trial 98 finished with value: 0.965034965034965 and parameters: {'lambda_l1': 1.6632496335962662e-05, 'lambda_l2': 1.5651151154213713e-07, 'num_leaves': 106, 'feature_fraction': 0.5546696995988858, 'bagging_fraction': 0.902063680583127, 'bagging_freq': 1, 'min_child_samples': 5}. Best is trial 39 with value: 1.0.


Did not meet early stopping. Best iteration is:
[96]	valid_0's binary_logloss: 0.0357584
Training until validation scores don't improve for 200 rounds
Did not meet early stopping. Best iteration is:
[56]	valid_0's binary_logloss: 0.0932818
Training until validation scores don't improve for 200 rounds


[I 2022-06-16 18:56:43,665] Trial 99 finished with value: 0.965034965034965 and parameters: {'lambda_l1': 1.0017760796268592e-05, 'lambda_l2': 1.1773897865611803e-08, 'num_leaves': 100, 'feature_fraction': 0.5296017934835784, 'bagging_fraction': 0.918681356342646, 'bagging_freq': 1, 'min_child_samples': 11}. Best is trial 39 with value: 1.0.


Did not meet early stopping. Best iteration is:
[47]	valid_0's binary_logloss: 0.0863906


In [5]:
# 最优结果
print(study.best_value)

# 最优参数
print(study.best_params)

1.0
{'lambda_l1': 0.014801116140580875, 'lambda_l2': 0.00011795510267537172, 'num_leaves': 203, 'feature_fraction': 0.950764211914787, 'bagging_fraction': 0.8553934254239814, 'bagging_freq': 1, 'min_child_samples': 43}
